<div style="background-color:#000;"><img src="pqn.png"></img></div>

We use yFinance to download historical price data, pandas for working with tables of data, numpy for numerical operations, and vectorbt for building and testing trading strategies.

## Imports and setup

This cell loads the required libraries and sets the start and end dates, as well as the ticker symbols for the two assets.

In [ ]:
import numpy as np
import pandas as pd
import vectorbt as vbt
import yfinance as yf

In [ ]:
start_date = "2010-01-01"
end_date = pd.Timestamp.today().strftime("%Y-%m-%d")
symbols = ["TLT", "SPY"]

With these imports and settings, we have the tools to download price data for TLT (long-term US Treasuries) and SPY (S&P 500) and work with it in the cells that follow. The tickers and date range are ready for further use.

## Download and organize price data

This cell downloads daily closing price data for TLT and SPY using yFinance and ensures all missing data is removed.

In [ ]:
price_data = yf.download(symbols, start=start_date, end=end_date, auto_adjust=False)[
    "Close"
]
price_data = price_data.dropna()

Here, we use yFinance to fetch historical closing prices for both tickers over our chosen period. The code organizes this data into a table and then filters out any dates that are missing information for either TLT or SPY. This guarantees that any further calculations use complete, reliable data on both assets.

This cell extracts date information and groups it by month to find key trading days.

In [ ]:
dates = price_data.index
year_month = price_data.index.to_period("M")
grouped = pd.DataFrame({"dt": dates, "ym": year_month}).groupby("ym")["dt"]
first_trading_days = grouped.first().values
last_trading_days = grouped.last().values

The code pulls out the trading dates and arranges them on a monthly calendar. It then figures out which day is the first and last trading date of each month. This organization helps us mark important points in each month that we can use to define trade entry and exit points later on.

## Create trading signal events

This cell defines two functions that help us find trading dates before or after certain monthly milestones.

In [ ]:
def get_prev_trading_day_idx(trading_dates, base_dates, offset):
    idx = []
    trading_dates = pd.Series(trading_dates)
    for d in base_dates:
        pos = trading_dates.searchsorted(d)
        prev_idx = pos - offset
        if prev_idx >= 0:
            idx.append(trading_dates.iloc[prev_idx])
    return pd.DatetimeIndex(idx)

In [ ]:
def get_offset_trading_day_idx(trading_dates, base_dates, offset):
    idx = []
    trading_dates = pd.Series(trading_dates)
    for d in base_dates:
        pos = trading_dates.searchsorted(d)
        target_idx = pos + offset
        if target_idx < len(trading_dates):
            idx.append(trading_dates.iloc[target_idx])
    return pd.DatetimeIndex(idx)

These two small tools let us find trading days just before or after specific dates. The first one moves backward from a list of monthly dates, returning dates a set number of days before. The second one moves forward by a set offset. This gives flexibility to mark and track entries and exits around key dates for our trading strategy.

This cell calculates special dates like seven days before the end of each month, the first trading day of each new month, and one week after that.

In [ ]:
pre_end_idx = get_prev_trading_day_idx(dates, last_trading_days, 7)
month_start_idx = get_offset_trading_day_idx(dates, last_trading_days, 1)
week_after_start_idx = get_offset_trading_day_idx(dates, month_start_idx, 7)

By picking out dates just before month-end, right at the start of the month, and a week into the following month, we set up a schedule for trading decisions. These timing markers get used to assign buy and sell moments for both TLT and SPY, setting the stage for later trades.

This cell sets up blank templates with all dates for tracking trade entry and exit signals for both long and short positions on each asset.

In [ ]:
entries_long_tlt = pd.Series(False, index=dates)
entries_short_spy = pd.Series(False, index=dates)
exits_long_tlt = pd.Series(False, index=dates)
exits_short_spy = pd.Series(False, index=dates)

In [ ]:
entries_short_tlt = pd.Series(False, index=dates)
entries_long_spy = pd.Series(False, index=dates)
exits_short_tlt = pd.Series(False, index=dates)
exits_long_spy = pd.Series(False, index=dates)

With these arrays, we set up a way to record trade events for buying or selling TLT and SPY, both in the normal direction (long) and the reverse (short). Every possible trading day is included, and we fill in the true signals for each position in later steps.

This cell fills in our trading signal templates: opening long and short trades at the planned times.

In [ ]:
entries_long_tlt.loc[pre_end_idx] = True
entries_short_tlt.loc[month_start_idx] = True

In [ ]:
entries_long_spy.loc[pre_end_idx] = True
entries_short_spy.loc[month_start_idx] = True

Here, we signal entries for long trades on both assets a week before month-end and entries for short trades on the first day of the new month. This mirrors a pattern that may take advantage of cyclical market movements, using both TLT and SPY in tandem.

This cell fills in our templates for when we will close out long and short trades.

In [ ]:
exits_long_tlt.loc[month_start_idx] = True
exits_short_tlt.loc[week_after_start_idx] = True

In [ ]:
exits_long_spy.loc[month_start_idx] = True
exits_short_spy.loc[week_after_start_idx] = True

For both TLT and SPY, we set exits for long trades on the first day of the month and close short positions one week after initiating them. This splits the strategy's open periods across key monthly markers, creating a consistent trading rhythm.

This cell organizes all trading signals into a clear table, making further analysis more straightforward.

In [ ]:
signals = pd.DataFrame(
    index=dates,
    columns=pd.MultiIndex.from_product(
        [symbols, ["long_entry", "long_exit", "short_entry", "short_exit"]]
    ),
    data=False,
)

In [ ]:
signals[("TLT", "long_entry")] = entries_long_tlt
signals[("TLT", "long_exit")] = exits_long_tlt
signals[("TLT", "short_entry")] = entries_short_tlt
signals[("TLT", "short_exit")] = exits_short_tlt
signals[("SPY", "long_entry")] = entries_long_spy
signals[("SPY", "long_exit")] = exits_long_spy
signals[("SPY", "short_entry")] = entries_short_spy
signals[("SPY", "short_exit")] = exits_short_spy

In [ ]:
long_entry = pd.DataFrame(
    {symbol: signals[(symbol, "long_entry")] for symbol in symbols}
)
long_exit = pd.DataFrame({symbol: signals[(symbol, "long_exit")] for symbol in symbols})
short_entry = pd.DataFrame(
    {symbol: signals[(symbol, "short_entry")] for symbol in symbols}
)
short_exit = pd.DataFrame(
    {symbol: signals[(symbol, "short_exit")] for symbol in symbols}
)

All our trading signals are now collected into a well-labeled table, with separate columns for entries and exits on both tickers and both long and short directions. Breaking out these entry and exit points into their own tables keeps things neat and ready for handoff to the trading engine.

## Test the trading strategy

This cell runs our strategy through the vectorbt engine to simulate performance, then summarizes key portfolio statistics.

In [ ]:
import numpy as np

In [ ]:
pf = vbt.Portfolio.from_signals(
    price_data,
    entries=long_entry,
    exits=long_exit,
    short_entries=short_entry,
    short_exits=short_exit,
    freq="D",
    size_type=1,
    size=np.inf,
    init_cash=100_000,
)

In [ ]:
pf.stats()

The code hands the historical price series and our carefully-timed trading signals to vectorbt, which runs a backtest over the whole period. It sets the initial cash to $100,000 and assumes trades happen in unlimited size to focus on signal logic. The strategy summary at the end delivers a breakdown of performance including return, risk, and other useful metrics, letting us judge how well this trading approach worked.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.